In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python310\python.exe -m pip install --upgrade pip' command.


In [22]:
import cv2
import os
import numpy as np

# to the left of the tape the luminance should be 1.25

################# VARIABLES TO BE USED #####################
# N_d: Digital number (value) of the pixel in the image
# t: the camera exposure time in seconds
# f_s: Aperture number (f-stop)
# S: ISO Sensitivity of the film
# L_s: Luminance of the scene, candela/meter^2

""" # number of bits in binary numbers
B = 8

# pixel value 
N_max = 2**B - 1 """

def calculate_calibration_constant(f_s, N_d, t, S, L_s):
    # a function for calculating the calibration constant
    K_c = (f_s**2 * N_d)/(t*S*L_s)
    return K_c

def calculate_luminance(f_s, N_d, t, S, K_c):
    # a function for calculating the luminance
    L_s = (f_s**2 * N_d)/(K_c*t*S)
    return L_s

def calculate_luminance_for_pixel(pixel, f_s, t, S, K_c):
    N_d = calculate_digital_number(pixel[0], pixel[1], pixel[2])
    return calculate_luminance(f_s, N_d, t, S, K_c)

def calculate_digital_number(R, G, B):
    # a function for calculating N_d value from RGB-values
    N_d = 0.2126*R + 0.7152*G + 0.0722*B
    return N_d

def calculate_middle_pixel_Nd(image):
    # a function for calculating N_d value for center pixel from image

    # accessing the center pixel
    center_x = image.shape[1] // 2
    center_y = image.shape[0] // 2
    center_pixel = image[center_x][center_y]

    # obtaining RGB-values for pixel
    R = center_pixel[0]
    G = center_pixel[1]
    B = center_pixel[2]

    return calculate_digital_number(R, G, B)

def calculate_median_Nd(image):
    # function for calculating the N_d value of the median of the middle 100x100 pixels

    # accessing the 100x100 pixels in the middle
    center_x = image.shape[1] // 2
    center_y = image.shape[0] // 2
    center_patch = image[center_y - 50:center_y + 50, center_x - 50:center_x + 50]

    # display the 100x100 pixel patch
    cv2.imshow('Center patch', center_patch)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Flatten the region into a list of RGB values
    pixels = center_patch.reshape(-1, 3)

    # Calculate the median value for each color channel
    median_values = np.median(pixels, axis=0)

    # Convert the median values to integers
    median_values = np.uint8(median_values)

    #print("Median RGB value:", median_values)

    # obtaining RGB-values for pixel
    R = median_values[0]
    G = median_values[1]
    B = median_values[2]

    return calculate_digital_number(R, G, B)

def calculate_luminances_for_image(ref_image, f_s, t, S, L_s):

    width = ref_image.shape[0]
    height = ref_image.shape[1]

    median_Nd = calculate_median_Nd(ref_image)

    # determine the calibration constant
    Kc_median = calculate_calibration_constant(f_s, median_Nd, t, S, L_s)

    luminances = [[0 for x in range(width)] for y in range(height)]

    for i in range(0,width-1):
        for j in range(0,height-1):
            pixel = ref_image[i][j]
            luminances[i][j] = calculate_luminance_for_pixel(pixel, f_s, t, S, Kc_median)


def main():

    # Camera variables
    t = 1/1.67
    f_s = 2.8
    S = 400
    L_s = 1.9          # Adjust to the known luminance (candela/meter^2)

    # Load the image
    image_path = os.path.join(os.getcwd(), 'images', 'tape_zoom.jpg')
    image = cv2.imread(image_path)

    if image is None:
        print("Error: Unable to load image. Please check the file path.")
    else:

        # Convert the image to 16-bit depth
        image_float = image.astype(np.float32)
        image_16bit = (image_float / 255.0) * 65535.0
        image_16bit = np.clip(image_16bit, 0, 65535).astype(np.uint16)
        
        # Display the 16-bit depth image
        cv2.imshow('16-bit Depth Image', image_16bit)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        # Convert the image to grayscale
        #gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # testing accessing one pixel
        """ pixel = image[0][5]
        print(pixel)
        pixel_Nd = calculate_digital_number(pixel[0], pixel[1], pixel[2])
        print(pixel_Nd) """

        # calculating Nd for middle pixel
        middle_pixel_Nd = calculate_middle_pixel_Nd(image)
        print('Middle pixel Nd:', middle_pixel_Nd)

        # calculating Nd for median pixel from 100x100 center pixels
        median_Nd = calculate_median_Nd(image)
        print('Median Nd: ', median_Nd)

        # calculating calibration constant using pixel in the middle
        Kc_middle = calculate_calibration_constant(f_s, middle_pixel_Nd, t, S, L_s)
        print('Calibration constant with middle pixel:', Kc_middle)

        # calculating calibration constant using median
        Kc_median = calculate_calibration_constant(f_s, median_Nd, t, S, L_s)
        print('Calibration constant with median:', Kc_median)

        height, width, channels = image.shape
        x = width // 2
        y = height // 2

        # calculating luminance in the middle of the image
        center_pixel = image[x,y]
        center_luminance = calculate_luminance_for_pixel(center_pixel, f_s, t, S, Kc_median)
        print('Luminance at the center of the image: ', center_luminance)

        # calculate luminance left of the tape
        # luminance should be about 1.25

        x = width // 4
        y = height // 2

        pixel_value = image[y, x]
        #print(pixel_value)

        pixel_Nd = calculate_digital_number(pixel_value[0], pixel_value[1], pixel_value[2])

        # calculating the luminance
        left_luminance = calculate_luminance(f_s, pixel_Nd, t, S, Kc_middle)
        print("Lumiance on the left of tape: ", left_luminance)

        calculate_luminances_for_image(image, f_s, t, S, L_s)
        
        """ # Calculate luminance (mean intensity) 
        luminance = gray_image.mean() 
        
        print("Luminance:", luminance)"""

if __name__ == '__main__':
    main()
    
    

Middle pixel Nd: 126.86139999999999
Median Nd:  90.0728
Calibration constant with middle pixel: 2.1854880762105258
Calibration constant with median: 1.551717310315789
Luminance at the center of the image:  2.6760205078558674
Lumiance on the left of tape:  1.3469931752290294
